<img src="images/strathsdr_banner.png" align="left">

# Spectrum Mapping Tool
----

<div class="alert alert-box alert-info">
Please use Jupyter Labs http://board_ip_address/lab for this notebook.
</div>

This notebook presents a spectrum map tool for PYNQ enabled devices. The user can interact with Python APIs for downloading, filtering, sorting, processing, and plotting the spectrum map. These APIs can be used to integrate the spectrum map into user applications.

## Table of Contents
* [Download the Spectrum Map](#download-the-spectrum-map)
* [Filter the Spectrum Map](#filter-spectrum-map)
* [Plot the Spectrum Map](#plot-the-spectrum-map)
* [Conclusion](#conclusion)

## References
* [OFCOM Spectrum Map for the UK](http://static.ofcom.org.uk/static/spectrum/map.html)

## Revision History
* **v1.0** | 11/09/2021 | Spectrum map tool notebook first version.

----

# Download the Spectrum Map <a class="anchor" id="download-the-spectrum-map"></a>
The UK spectrum map source can be downloaded from OFCOM using the code cell below. The source is stored in a folder named 'spectrum' in the same directory as the source code.

In [ ]:
from pynq_specmap import application, download, filters, plots

download.download_bands()

The 'spectrum' folder will contain a spectrum map object named 'bands.pickle'. This is created when the `refresh_bands_object` function is called in the cell below. Without filtering arguments, this function extracts the most recently downloaded spectrum map, applies user defined filtering, stores it as 'bands.pickle', and finally returns the bands to the user.

In [ ]:
def filter_callback(bands):
    """Standard filter for Gen 1 RFSoC
    and UK Spectrum Map data from OFCOM.
    
    """
    bands = filters.filter_bands_range(bands, uf=4096e6)
    bands = filters.filter_bands(bands, s=['N/A', ''], include=False)
    bands = filters.delete_bands_duplicate(bands)
    return bands

%time bands = download.refresh_bands_object(filter_callback=filter_callback)

Finally, if you would like to retrieve the most recent spectrum map object after a system reboot, just use `retrieve_bands_object`. This will load the previously filtered spectrum map contained in 'bands.pickle'. This method may be useful if a lot of filtering has been applied to remove unwanted bands.

In [ ]:
%time bands = download.retrieve_bands_object()

Now we can look at a band. Notice that the band is a pandas data frame containing seven columns:
* lf — The lower frequency cutoff
* uf — The upper frequency cutoff
* bandwidth — The bandwidth of the lower and upper frequencies
* s  — The band sector name
* u  — The unique band name
* v  — It is not apparant what this is...
* trace — A plotly graphic object scatter trace for plotting

In [ ]:
bands.head()

Lastly, it is possible to retrieve the filenames in the 'spectrum' folder containing previously downloaded spectrum maps. Use `get_bands_filename(region='', date=None, time=None)` to retrieve the filenames, where region, date, and time can be used to filter the filenames. 

In [ ]:
download.get_bands_filename()

Naming convention of the file is required, where the spectrum map will be allocated a region, date, and time.
For example, a file name may be 'spectrum_uk_20210906_155255.json', where spectrum indicates that the file contains a spectrum map, uk indicates the region, 20210906 is the %YYYY%MM%DD, and 155255 is the %HH%MM%SS.

The most recent spectrum map can be given by slicing the return value of `get_bands_filename`.

In [ ]:
download.get_bands_filename()[-1]

----

# Filter the Spectrum Map <a class="anchor" id="filter-the-spectrum-map"></a>
The possible values across all bands can be extracted using `get_bands_unique_values`. This function will extract all possible values once, and append them to a list. The list is combined with an appropriate key to form a dictionary. The code cell below will present the band sector names found across all bands. You can change the code cell to present the results from other keys such as, lf, uf, bandwidth, u, v, and trace.

In [ ]:
filters.get_bands_unique_values(bands)['s']

Using the band sector names above, we can extract unique bands in the system. Lets use `filter_bands` to easily extract the Mobile and Wireless Broadband bands.

In [ ]:
wireless_bands = filters.filter_bands(bands, s=['Wireless Broadband'])
wireless_bands.append(filters.filter_bands(bands, s=['Mobile']))

There are still quite a significant number of bands belonging to the Mobile and Wireless Broadband subset. Running the code cell below will print the number of bands in the subset.

In [ ]:
print('Number of mobile and wireless broadband bands:', len(wireless_bands))

Bands that meet a particular frequency range can be easily filtered using `filter_bands_range`. The code cell below extracts the downlink and uplink bands for mobile spectrum access at 800MHz. The unique band names and their cutoff frequencies are then printed for clarity. We should be able to see the downlink and uplink spectrum access for 3G, EE, Vodafone, and Telefonica.

In [ ]:
mobile_800MHz_bands = filters.filter_bands_range(wireless_bands, lf=370e6, uf=380e7)
for _, band in mobile_800MHz_bands.iterrows():
    print('Band name: ', band['u'], \
          '\r\nLower frequency: ', band['lf'], \
          '\r\nUpper frequency: ', band['uf'], '\r\n')

Now that the downlink and uplink mobile bands have been acquired, we can inspect the bandwidth of each band.

In [ ]:
mobile_800MHz_bands

The bands can be sorted so that they are now in order of bandwidth.

In [ ]:
mobile_800MHz_bands.sort_values('bandwidth')

If we only want to inspect bandwidths of 10MHz or more, we can filter the bands using `filter_bands_bandwidth` to extract the bands of interest.

In [ ]:
filtered_800MHz_bands = filters.filter_bands_bandwidth(mobile_800MHz_bands, lbw=10e6)
filtered_800MHz_bands

----

# Plot the Spectrum Map <a class="anchor" id="plot-the-spectrum-map"></a>
There are a large number of bands in the spectrum map. For instance, lets look only at the mobile bands around 800MHz only for now.

In [ ]:
print('Number of mobile and wireless broadband bands:', len(mobile_800MHz_bands))

To plot the spectrum map, the number of required traces can be reduced by merging neighbouring bands together to form one trace. This is achieved using `merge_bands_threshold(bands, threshold=0, unique=False)`. The method uses a threshold argument to merge bands based on their distance from one another. We will keep this as 0 for now. The unique argument, if True, will merge bands so long as their unique names are the same. Lets try it on the mobile bands around 800MHz and print the resulting bands after merging.

In [ ]:
mobile_800MHz_bands_merged = filters.merge_bands_threshold(mobile_800MHz_bands)
mobile_800MHz_bands_merged

As shown, after merging, the bands have combined to form two traces, which is easier to plot later using the spectrum map tool. To ensure that merging completed successfully lets look at the unique name of the downlink band and the unique name of the uplink band.

In [ ]:
from IPython.core.display import display, HTML

for index, band in mobile_800MHz_bands_merged.iterrows():
    display(HTML(''.join(['<b>Band: ', str(index), '</b><br>', band['u']])))

The remaining bands did not merge into one as these did not meet the threshold criteria. We can use the merge bands technique to reduce the number of traces that need to be drawn to the screen at one. This saves processing time and is a more user friendly experience.

Finally, lets now build the spectrum map tool. This tool uses filtering, merging, and plotting to display the spectrum map conveniently to the user.

In [ ]:
application.spectrum_map_tool(bands, template='plotly_dark').children[1]

Use the plot to explore the spectrum map, or navigate the different bands using the control interface on the right side of the screen. Using the dropdown, the spectrum sector can be changed. The spectrum bands can be selected using the selector tool.

----

# Conclusion
This notebook has presented Python methods and functions to download and process the spectrum map provided by the UK regulators, OFCOM. The API presented in this notebook was used to construct a spectrum map tool, where spectrum sectors could be plotted and evaluated by the user in real-time.